LIGHTGBM REGRESSOR

In [3]:

!pip install lightgbm


  Using cached lightgbm-4.6.0-py3-none-win_amd64.whl.metadata (17 kB)
Using cached lightgbm-4.6.0-py3-none-win_amd64.whl (1.5 MB)


In [3]:
#IMPORTING LIBRARIES
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error


In [25]:
from lightgbm import LGBMRegressor

In [4]:
#LOADING DATA
data = pd.read_excel('C:/Users/sahan/OneDrive/Desktop/demo python code/MACHINE HEALTH & MAINTENANCE.xlsx')

In [7]:
print("Shape of Dataset:",data.shape)

Shape of Dataset: (20000, 10)


In [10]:
print(data.head(5))

            Timestamp  MachineID    Plant  Temperature  Vibration   Pressure  \
0 2025-01-01 00:00:00        137  Plant_C    69.707421   3.937962  29.726777   
1 2025-01-01 01:00:00        143  Plant_C    68.929016   4.891410  33.855301   
2 2025-01-01 02:00:00        112  Plant_C    81.257229   5.399394  22.459834   
3 2025-01-01 03:00:00        108  Plant_A    86.751817   2.759409  36.211147   
4 2025-01-01 04:00:00        109  Plant_B    78.711891   6.594050  26.540506   

   EnergyConsumption  ProductionUnits  DefectCount  MaintenanceFlag  
0         254.562963              133            2                0  
1         257.151463              155            2                0  
2         274.710842              119            4                0  
3         277.620583              120            2                0  
4         199.763661              177            5                0  


In [11]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Timestamp          20000 non-null  datetime64[ns]
 1   MachineID          20000 non-null  int64         
 2   Plant              20000 non-null  object        
 3   Temperature        20000 non-null  float64       
 4   Vibration          20000 non-null  float64       
 5   Pressure           20000 non-null  float64       
 6   EnergyConsumption  20000 non-null  float64       
 7   ProductionUnits    20000 non-null  int64         
 8   DefectCount        20000 non-null  int64         
 9   MaintenanceFlag    20000 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(4), object(1)
memory usage: 1.5+ MB
None


In [13]:
data.describe(include='all')

,Timestamp,MachineID,Plant,Temperature,Vibration,Pressure,EnergyConsumption,ProductionUnits,DefectCount,MaintenanceFlag
count,20000,20000.000000,20000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
unique,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,Plant_B,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,6822,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,2026-02-21 15:30:00,124.358750,NaN,69.947559,4.995461,29.993698,250.189198,124.649700,2.993550,0.101600
min,2025-01-01 00:00:00,100.000000,NaN,29.895390,-1.152912,6.456349,86.660670,50.000000,0.000000,0.000000
25%,2025-07-28 07:45:00,112.000000,NaN,63.236121,3.970814,26.626777,223.099989,87.000000,2.000000,0.000000
50%,2026-02-21 15:30:00,124.000000,NaN,69.940758,4.987013,30.013254,250.051619,125.000000,3.000000,0.000000
75%,2026-09-17 23:15:00,137.000000,NaN,76.664978,6.024795,33.391801,277.047594,162.000000,4.000000,0.000000
max,2027-04-14 07:00:00,149.000000,NaN,109.305828,11.558926,50.295742,426.441256,199.000000,13.000000,1.000000


In [14]:
data.isnull().sum()

Timestamp            0
MachineID            0
Plant                0
Temperature          0
Vibration            0
Pressure             0
EnergyConsumption    0
ProductionUnits      0
DefectCount          0
MaintenanceFlag      0
dtype: int64

In [16]:
data.columns = data.columns.str.lower()
data.columns

Index(['timestamp', 'machineid', 'plant', 'temperature', 'vibration',
       'pressure', 'energyconsumption', 'productionunits', 'defectcount',
       'maintenanceflag'],
      dtype='object')

In [7]:
#cleaning data and make the dataset to adapt real data
data = data.dropna()


In [8]:
data.drop_duplicates()

,Timestamp,MachineID,Plant,Temperature,Vibration,Pressure,EnergyConsumption,ProductionUnits,DefectCount,MaintenanceFlag
0,2025-01-01 00:00:00,137,Plant_C,69.707421,3.937962,29.726777,254.562963,133,2,0
1,2025-01-01 01:00:00,143,Plant_C,68.929016,4.891410,33.855301,257.151463,155,2,0
2,2025-01-01 02:00:00,112,Plant_C,81.257229,5.399394,22.459834,274.710842,119,4,0
3,2025-01-01 03:00:00,108,Plant_A,86.751817,2.759409,36.211147,277.620583,120,2,0
4,2025-01-01 04:00:00,109,Plant_B,78.711891,6.594050,26.540506,199.763661,177,5,0
...,...,...,...,...,...,...,...,...,...,...
19995,2027-04-14 03:00:00,115,Plant_A,74.107527,5.381079,31.456000,294.013239,57,2,0
19996,2027-04-14 04:00:00,111,Plant_A,65.328735,6.964223,26.614313,231.190106,150,6,0
19997,2027-04-14 05:00:00,125,Plant_A,60.216298,3.429051,38.605298,270.326239,180,4,1
19998,2027-04-14 06:00:00,126,Plant_B,59.320378,3.575870,28.603057,257.354853,155,7,0


In [19]:
#Defining features and traget variable
features = [
    'temperature',
    'vibration',
    'pressure',
    'energyconsumption',
    'productionunits',
    'plant',
    'machineid'
]
target = 'defectcount'

In [20]:
X = data[features].copy()
y = data[target].copy()

In [21]:
#training testing 
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.15,random_state=42)

In [22]:
#preprocessing the data
cat_features = ['plant','machineid']
num_features = [c for c in features if c not in cat_features]

In [23]:
preprocessor = ColumnTransformer([
    ('num',Pipeline([
        ('impute',SimpleImputer(strategy='median')),
        ('scale', StandardScaler())
    ]),num_features),
], remainder='drop')

In [31]:
!pip install catboost
from catboost import CatBoostRegressor

   ---------------------------------------- 0.0/102.4 MB ? eta -:--:--
   ---------------------------------------- 0.3/102.4 MB ? eta -:--:--
   ---------------------------------------- 1.0/102.4 MB 3.1 MB/s eta 0:00:33
    --------------------------------------- 2.4/102.4 MB 4.3 MB/s eta 0:00:24
   - -------------------------------------- 3.7/102.4 MB 5.0 MB/s eta 0:00:20
   - -------------------------------------- 3.9/102.4 MB 4.9 MB/s eta 0:00:21
   - -------------------------------------- 4.5/102.4 MB 3.8 MB/s eta 0:00:26
   -- ------------------------------------- 5.8/102.4 MB 4.1 MB/s eta 0:00:24
   -- ------------------------------------- 7.1/102.4 MB 4.4 MB/s eta 0:00:22
   --- ------------------------------------ 8.7/102.4 MB 4.7 MB/s eta 0:00:21
   --- ------------------------------------ 10.2/102.4 MB 4.9 MB/s eta 0:00:19
   ---- ----------------------------------- 12.6/102.4 MB 5.5 MB/s eta 0:00:17
   ----- ---------------------------------- 14.7/102.4 MB 5.8 MB/s eta 0:00:

In [33]:
model = CatBoostRegressor(
    iterations=1500,  # like n_estimators
    learning_rate=0.03,
    depth=7,  # like max_depth
    loss_function='Poisson',  # or 'Tweedie:variance_power=1.5' for more flexibility
    eval_metric='MAE',
    random_seed=42,
    verbose=100  # prints progress
)

In [ ]:
#choose model
"""model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LGBMRegressor(
        n_estimators=1200,
        learning_rate=0.035,
        max_depth=8,
        num_leaves=60,
        min_child_samples=40,
        subsample=0.85,
        colsample_bytree=0.75,
        reg_lambda=0.5,
        random_state=42,
        verbose=-1
    ))
])"""

In [34]:
# Fit with categorical indices
cat_idx = [features.index('plant'), features.index('machineid')]
model.fit(X_train, y_train, cat_features=cat_idx, eval_set=(X_test, y_test), early_stopping_rounds=60)

0:	learn: 2.9537899	test: 3.0220745	best: 3.0220745 (0)	total: 228ms	remaining: 5m 41s
100:	learn: 2.0594303	test: 2.1131700	best: 2.1131700 (100)	total: 3.97s	remaining: 55s
200:	learn: 2.0295588	test: 2.0827081	best: 2.0827081 (200)	total: 8.84s	remaining: 57.1s
300:	learn: 2.0263878	test: 2.0805050	best: 2.0805050 (300)	total: 14.2s	remaining: 56.6s
400:	learn: 2.0235916	test: 2.0799933	best: 2.0797969 (363)	total: 20.5s	remaining: 56.1s
Stopped by overfitting detector  (60 iterations wait)

bestTest = 2.079796937
bestIteration = 363

Shrink model to first 364 iterations.


In [35]:
y_pred = model.predict(X_test)

In [27]:
#Train and Evaluate the model
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

c:\Users\sahan\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [36]:
#calculating metrics
mae = mean_absolute_error(y_test,y_pred)
rmse = np.sqrt(mean_squared_error(y_test,y_pred))

In [38]:
print(f"MAE = {mae:.4f}")
print(f"RMSE = {rmse:.4f}")

MAE = 1.3627
RMSE = 1.7317
